# Fig 2: Dimensionless multiplicity function $M^2 n(M)/\bar\rho$ and linear halo bias $b(M)$

First we import the libraries we need:|

In [ ]:
# Standard imports
import sys
import numpy as np
import matplotlib.pyplot as plt

# Third-party imports
import camb # Calculates the linear matter power spectrum

# Halo Model library imports
sys.path.append('../halomodel/')
import cosmology
import halomodel

Now we set the comsological parameters. If `sigma_8_set = True` we scale the linear power spectrum to account for the new sigma_8 value.  

In [ ]:
# Set cosmological parameters
Omega_c = 0.25
Omega_b = 0.05
Omega_k = 0.0
h = 0.7
As = 1.97448e-9
ns = 0.96
w = -1.0
wa = 0.0
m_nu = 0.0 # in eV

# You can choose to set sigma_8, in that case we scale the power spectrum
sigma_8_set = True # If true uses the following value
sigma_8_in  = 0.7

Now let's set some parameters that we will use throughout the calculation: First, we set a range of wavenumbers, `k`, and then fill array, `ks`, we then set a redshift, `z`.

In [ ]:
# k range [h/Mpc]
kmin = 1e-3; kmax = 200
nk = 101
ks = np.logspace(np.log10(kmin), np.log10(kmax), nk)

# Starting redshift
z = 0.
zmax = 2.

# Redshift
zs = [1., 0.] # CAMB reorders from high-z to low-z, so we define it like this from the start to avoid confusion

Now we initialise `CAMB` and produce a linear matter power spectrum. We have the option here to scale the power by an input `sigma_8` value: `sigma_8_in`

In [ ]:
# Sets cosmological parameters in camb to calculate the linear power spectrum
pars = camb.CAMBparams()
wb   = Omega_b*h**2
wc   = Omega_c*h**2

# This function sets standard and helium set using BBN consistency
pars.set_cosmology(ombh2=wb, omch2=wc, H0=100.*h, mnu=m_nu, omk=Omega_k)
pars.set_dark_energy(w=w, wa=wa, dark_energy_model='ppf') 
pars.InitPower.set_params(As=As, ns=ns, r=0)
pars.set_matter_power(redshifts=zs, kmax=kmax) # Setup the linear matter power spectrum

# extract parameters from CAMB
Omega_m  = pars.omegam 
Omega_nu = pars.omeganu
Omega_L  = 1. + Omega_k - Omega_m
camb_results = camb.get_results(pars)
sigma_8 = (camb_results.get_sigma8()[zs.index(z)]).item()

if sigma_8_set:
    scaling = (sigma_8_in/sigma_8)**2
    As *= scaling
    pars.InitPower.set_params(As=As, ns=ns, r=0)

Pk_lin = camb.get_matter_power_interpolator(pars, 
                                            nonlinear = False, 
                                            hubble_units = True, 
                                            k_hunit = True, 
                                            kmax = kmax,
                                            var1 = camb.model.Transfer_tot,
                                            var2 = camb.model.Transfer_tot, 
                                            zmax = zmax,
                                           )
Pk_lin = Pk_lin.P # Single out the linear P(k) interpolator
camb_results = camb.get_results(pars)
sigma_8 = (camb_results.get_sigma8()[zs.index(z)]).item()

Now we set a range of halo masses, `M`, and fill an array, `Ms`. This halo-mass range needs to be wide enough that it includes all 'interesting' halo masses from the point of view of the calculation, and the mass-spacing needs to be fine enough that the calculation converges; in an actual use-case the effect on power spectra of both mass range and spacing should be convergence tested. Then we find the Lagrangian radii, `R`, corresponding the the halo masses.

In [ ]:
# Halo mass range [Msun/h]
Mmin = 1e9; Mmax = 1e15; nM = 129
Ms = np.logspace(np.log10(Mmin), np.log10(Mmax), nM)

# Lagrangian radii corresponding to halo masses; 
Rs = cosmology.Lagrangian_radius(Ms, Omega_m)

Next, we need to get an array of `sigma(R)` values. This requires an integral over the linear power spectrum times the Fourier transform of a top hat function, which is the oscillatory sinc function. We can use different integration methods to calculate this. Three methods are given: 
* `brute`: uses a brute force sum over `nk` log-bins between `kmin` and `kmax`
* `quad`: uses the scipy.integrate.quad routine integrates between zero and infinity
* `camb`: uses the internal camb integration

In [ ]:
# Get sigma(R) from CAMB
sigmaRs_camb = cosmology.get_sigmaR(Rs, camb_results, integration_type='camb')
sigmaRs = sigmaRs_camb

Now we can create a halo model, which has to be reinitialised at each different redshift of interest. In this example notebook we are only doing calculations at a single redshift, so we initialise the halo model with that in mind. We also need to choose a mass function and linear halo bias, in this example these come from `Tinker et al. (2010)`, and we need to choose a halo definition, here we choose `330`, so haloes are defined to be spherical objects that contain an average density that is 330 times greater than the mean background universe, which is approximately the virial definition for this cosmology at `z=0`.

In [ ]:
# Initialise halo model
hmod = halomodel.halo_model(z, Omega_m, hm='Tinker et al. (2010)', Dv=330.)

Now lets make figure 2: Dimensionless multiplicity function, $M^2 n(M)/\bar\rho$, as a function of halo mass for three popular halo mass functions: Sheth and Tormen 1999; Tinker et al. 2010; Despali et al. 2016 at z = 0 (solid) and z = 1 (dashed) for the virial halo definition. Note that the kmax value for Pk_lin is very important here. We chose kmax=200, which produces smooth functions. A smaller kmax can produce noisy results for lower masses.

In [ ]:
mass_functions = [
    'Sheth & Tormen (1999)',
    'Tinker et al. (2010)',
    'Despali et al. (2016)',
]

# Calculate b(M) and n(M)
# NOTE: Remove slow calculation with sigma(R) from P(k)
bs = []
Fs = []
for iz, z in enumerate(zs):
    for mass_function in mass_functions:
        print(mass_function, 'z = %1.1f'%(z))
        hmod = halomodel.halo_model(z, Omega_m, hm=mass_function, Dv=330.)
        b = hmod.linear_bias(Ms, sigmas=sigmaRs[iz])
        #F = halomodel.halo_multiplicity_function(hmod, Ms, Pk_lin=lambda k: Pk_lin(z, k))
        F = hmod.multiplicity_function(Ms, sigma=lambda R: camb_results.get_sigmaR(R, hubble_units=True, return_R_z=False)[iz])
        bs.append(b)
        Fs.append(F)

In [ ]:
# Make the plot
plt.subplots(2, 1, figsize=(5, 4), dpi=100, sharex=True)
n = len(mass_functions)

# Mass function
plt.subplot(2, 1, 1)
plt.plot(Ms[0], np.nan, color='black', ls='-', label=r'$z=0$')
plt.plot(Ms[0], np.nan, color='black', ls='--', label=r'$z=1$')
for i, mass_function in enumerate(mass_functions):
    plt.plot(Ms, Fs[i], ls='--', color='C%d'%i)
    plt.plot(Ms, Fs[i+n], ls='-', color='C%d'%i)
plt.xscale('log')
plt.gca().set_xticklabels([])
plt.ylim((0., 0.06))
plt.ylabel(r'$M^2 n(M)/\bar\rho$')
# Common x-axis
plt.xlim((Ms[0], Ms[-1]))
plt.legend()

# Linear bias
plt.subplot(2, 1, 2)
plt.axhline(1., color='black')
for i, mass_function in enumerate(mass_functions):
    plt.plot(Ms, bs[i], ls='--', color='C%d'%i)
    plt.plot(Ms, bs[i+n], ls='-', color='C%d'%i, label=mass_function)
plt.xlabel(r'$M / h^{-1} M_\odot$')
plt.xscale('log')
plt.ylim((0., 5.))
plt.ylabel(r'$b(M)$')

# Common x-axis
plt.xlim((Ms[0], Ms[-1]))
plt.legend()

plt.savefig('plots/hmf_and_bias.pdf',bbox_inches='tight')
plt.show()